In [ ]:
# https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f
# ID3

Attribute vs Measure
https://analystanswers.com/what-is-a-data-attribute-definition-types-examples/

https://github.com/DunZhang/DPTrees

In [381]:
import pandas as pd 
import numpy as np 
from collections import Counter, defaultdict
import random
import math

In [382]:
dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [383]:
random.seed(42)
adult2 = us_adult_income.sample(frac=0.05, random_state=42)
adult2.shape

(1628, 15)

In [384]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [385]:
adult23 = adult25[["Age", "Education", "Occupation", "Gender", "Income"]]
adult23

,Age,Education,Occupation,Gender,Income
14160,27,15,0,0,1
27048,45,11,3,0,1
28868,29,9,3,1,0
5667,30,9,6,0,1
7827,29,15,2,1,1
...,...,...,...,...,...
17491,19,11,11,0,1
18954,50,8,9,1,1
15112,44,11,2,1,1
5477,19,15,9,1,1


In [386]:
np.random.seed(97)
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

In [387]:
D4 = D2.astype(str)
D4

array([['50', '11', '3', '1', '0'],
       ['29', '11', '3', '1', '1'],
       ['24', '9', '9', '0', '1'],
       ...,
       ['47', '11', '0', '0', '1'],
       ['46', '11', '11', '0', '1'],
       ['55', '5', '13', '1', '1']], dtype='<U21')

In [388]:
# make sure both test and training data have all classes in them
# the code will not run if D88 and D99 do not print the same arrays
D22 = D2[400:]

In [389]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([-1,  0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [390]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([-1,  0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [391]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,1]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [392]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,1]).astype(int))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

GainRatio is used to select the node splitting attribute and the splitting point of the continuous attribute value in the C4.5 algorithm

A_i is an attribute in the attribute set A to be split, a_1 and a_2 are weight coefficients assigned to the information gain and the gainratio: 0 <= a_1, a_2 <= 1, and a_1 + a_2 = 1.

IG_GR(Data, A_i) = (a_1 + a_2 / SplitInformation(D, A_i)) * InfoGain(D, A_i)

##### C4.5 
https://github.com/barisesmer/C4.5/tree/master/c45

https://github.com/Valdecy/C4.5/blob/master/Python-DM-Classification-04-C4.5.py

https://github.com/loginaway/DecisionTree

In [393]:
import numpy as np

class Node(object):
    '''
    Basic element for decision tree.
    Initialization: to initialize a tree node, at least feature_name is needed.
    Connection: to connect different nodes, there are two situations.
        1, if the parent node is discrete (i.e. node.discrete==True), 
            then 
            >> node[feature_value]=childnode 
            will do the connection.
        2, if the parent node is continuous, then 
            >> node['<=']=childnode 
            or 
            >> node['>']=childnode 
            will assign its children. To be specific, 
            node['<=']=childnode means if the feature value is bigger than 
            node.threshold, then the workflow goes left to the childnode, and vice versa.
    
    Classification: if you have constructed a tree, you may want to classify a 
        certain group of data. Simply call the root node will do this task, i.e.
        >> root(data)
        where data should be a dictionary containing the corresponding key and value.
        And this method will return the classification result.

    Visualization: A naive visualization is implemented here. 
        For example, you may visualize a subtree rooted at 'node' by
        >> print(node)
        The tree structure will then be printed on the console.
    '''
    def __init__(self, feature_name='', discrete=True, threshold=0, depth=0, isLeaf=False, classification=None):
        # for discrete node: the next node can be retrieved by self.map[feature_value]
        # for continuous node: by self.map['<='] (when value <= self.threshold)
        #                       by self.map['>'] (when value > self.threshold)

        self.map=dict()

        self.discrete=discrete
        self.feature_name=feature_name
        self.isLeaf=isLeaf
        if isLeaf:
            self.classification=classification
        if not discrete:
            self.threshold=threshold
        self.depth=depth
        self._class_counts = defaultdict(int)
        self._noisy_class_counts = None

    def __setitem__(self, key, value):
        self.map[key]=value

    def __getitem__(self, key):
        return self.map.get(key)

    def __getdepth__(self):
        return self.depth 

    def increment_class_count(self, class_value):
        self._class_counts[str(class_value)] += 1

    def __noisy_class_counts__(self, epsilon, class_values):
        
        ''' Add noise to the class counts of this leaf. Not performed on parents. '''
        if not self._noisy_class_counts and not self._children: # to make sure this code is only run once per leaf
            
            counts = {}
            for val in class_values:

                if val in self._class_counts:
                    # Laplace noise added to THE CLASS COUNTS
                    counts[val] = max( 0, int(self._class_counts[val] + np.random.laplace(scale=float(1./epsilon))) )
                else: # original count was 0
                    counts[val] = max( 0, int(np.random.laplace(scale=float(1./epsilon))) )

            self._noisy_class_counts = counts

            # what I'm assuming is equation (10)
            self._signal_to_noise = (epsilon * 1.0 * sum([v for k,v in counts.items()])) / (math.sqrt(2*1.0) * len(counts)) # enS / (sqrt(2n) * num_classes)
         
            #print("self._signal_to_noise = {}".format(self._signal_to_noise))
            return 0
        else:
            return 1 # we're summing the redundant calls

    def __call__(self, data): # returns the classification result! 
        
        '''
        This method should be used on the root to predict its classification.
        data: a dict with its key being the features and value being 
        the corresponding value.
        '''

        # print("Inside __call__ method of Node")

        #print("data: ")
        #print(data)

        #print("feature name: ")
        #print(self.feature_name)

        #print("the map right now")
        #print(self.map)

        if self.isLeaf:
            return self.classification

        # if the node has a discrete feature, use __getitem__ to find the next node
        # then call the next node.
        if self.discrete:
            return self.map[data[self.feature_name]](data)

        else:
        # node is not discrete: print the self.feature_name
            #print("continuous name")
            #print(self.feature_name)

            if data[self.feature_name].astype(np.float32)>self.threshold:
                return self.map['>'](data)
            else:
                return self.map['<='](data)

    def __str__(self):
        '''
        This method is designed for printing the subtree rooted at the current node.
        To print the tree, try print(self).
        '''
        hierarchy={}
        stack=[(0, 0, self)]
        count=0
        while stack:
            # BFS
            (layer_index, node_index, current)=stack.pop(0)
            layer_index+=1

            hierarchy[layer_index]=hierarchy.get(layer_index, [])
            hierarchy[layer_index].append((node_index, current.feature_name,\
                    [(str(current.classification),)] if current.isLeaf else \
                     [(str(key), '' if current.discrete else str(current.threshold), item.feature_name) for key, item in current.map.items()]))
            # print(hierarchy)

            if current.isLeaf:
                continue
            for _, item in current.map.items():
                count+=1
                stack.append((layer_index, count, item))
            # print(item_index_map)

        totallist=[]
        for layer, layer_content in hierarchy.items():
            rowlist=[]
            for i in layer_content:
                rowlist.append('['+i[1]+']'+': {'+', '.join([' '.join(item) for item in i[2]])+'}')
            rowstr=' +++ '.join(rowlist)
            totallist.append(rowstr)

        return '============Root===========\n'+'\n\n'.join(totallist)+'\n============Leaf==========='

In [ ]:
#D6= np.genfromtxt('../datafiles/iris_train.txt', dtype=str)
'''
D6 = adult23.to_numpy()
D6 = D6.astype(str)
np.random.shuffle(D6)

k=len(D6)

#attribute_name=['sepal length', 'sepal width', 'petal length', 'petal width']

attribute_name = ["Age", "Education", "Occupation", "Gender"]

A={name: [i] for i, name in enumerate(attribute_name)}

print(A)

# feature_discrete = {name: False for name in attribute_name}
feature_discrete={name: False if isinstance(adult2[name].iloc[0], np.integer) else True for name in attribute_name}

conf={'A': A, 
      'feature_discrete': feature_discrete, 
      'treeType':'IGGR'}


dt=Tree_OLD(conf)

dt.train(D6[:len(D6)//2])
print(dt.tree)
print(dt.eval(D6[len(D6)//2:]))
dt.prune(D6[:len(D6)//2], D6[len(D6)//2:])
print(dt.tree)
print(dt.eval(D6[len(D6)//2:]))
'''

In [394]:
herlist = D2[:, 4]
unique, counts = np.unique(herlist, return_counts=True)
print(np.asarray((unique, counts)).T)

[['0' '399']
 ['1' '1229']]


In [395]:
np.unique(D2[:, 4].astype(float)> float(str(951.0)))

array([False])

In [396]:
D2[np.argwhere(D2[:, 4].astype(float) > 951.0), -1].flatten()

array([], dtype='<U21')

In [296]:
dm = 5
sum(2/(dm - i) for i in range(0, (dm - 2))) + (2 / (dm - (dm-1) + 2 )) + 1 # assuming we reach maximum depth

3.2333333333333334

In [397]:
# currently includes methods as:
        # _init_  : _trees
        # get_domains
        # reduce trees
        # evaluate_accuracy_with_voting

class DPRF_Forest:  
    def __init__(self, 
                 training_nonprocessed,
                 training_data, # 2D list of the training data where the columns are the attributes, and the first column is the class attribute
                 # test_data, # T
                 epsilon, # epsilon, Budget, B, the total privacy budget
                 f, # n of attributes to be split used by each dctree f
                 max_depth # max tree depth 
                 ):

        # Forest = {}
        self._trees = []

        self.epsilon = epsilon

        self.max_depth = max_depth


        ''' Some initialization '''
        # Attribute set F
        attribute_values = self.get_domains(training_data)

        # Class values set
        class_values = [str(y) for y in list(set([x[len(x)-1] for x in training_data]))]
        attribute_indexes = [int(k) for k,v in attribute_values.items()]


        #print(attribute_values)
        #print("------------")
        #print(class_values)
        #print("------------")
        #print(attribute_indexes)
        #print("-----------")

        
        # |D|
        dataset_size = len(training_data)


        valid_attribute_sizes = [[int(k),len(v)] for k,v in attribute_values.items()]

        # Number of trees t
        num_trees = len(attribute_indexes)
 

        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] )

        # e = B / t
        epsilon_per_tree = epsilon / float(num_trees)

        ''' Calculating minimum support threshold '''
        # estimated_support_min_depth = dataset_size / (average_domain**2) # a large number
        # estimated_support_max_depth = dataset_size / (average_domain ** (len(attribute_indexes)/2)) # max tree depth is k/2 # a small number
        # epsilon_per_tree = epsilon / float(num_trees)
        # min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER # the minimum support in order for S>N


        # while estimated_support_max_depth > min_support: # then we can have more trees
        #    num_trees += 1
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        # while min_support > estimated_support_min_depth and num_trees>1: # then we need to have less trees
        #    num_trees, valid_attribute_sizes, estimated_support_min_depth = self.reduce_trees(num_trees, valid_attribute_sizes, dataset_size)
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        print("NUM TREES = {} & EPSILON PER TREE = {}".format(num_trees, epsilon_per_tree))

        # ? 
        root_attributes = []

       
        for a in attribute_indexes:
            if a in [x[0] for x in valid_attribute_sizes]:
                root_attributes.append(a) # OR: [index, support, gini]

        print("root attributes")
        print(root_attributes)

        # for treeId = 1,2 ... t do:
        for t in range(num_trees):

            if not root_attributes:
                root_attributes = attribute_indexes[:]

            # randomly extract |D| samples from D w/ a bagging algo?
            root = random.choice(root_attributes)
            root_attributes.remove(root)

            print("what is root attributes: ")
            print(root_attributes)


            # TREE BUILDTREE(D, A, C, eps, f, Dm)
            # D :

            # attribute_indexes, - A_ind
            # attribute_values, A - not exactly the same after all! 
            # root, 
            # class_values, 
            # feature_discrete
            # treetype 
            # dataset_size, 
            # epsilon_per_tree

            A={name: [i] for i, name in enumerate(adult23.columns[:-1])}

            # {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True, 'CapitalLoss': False}
            feature_discrete={name: False if isinstance(training_nonprocessed[name].iloc[0], np.integer) else True for name in adult23.columns[:-1]}

            #print("feature discrete")
            #print(feature_discrete)

            #print("---------------")
            #print("A: ")
            #print(A)
            #print("---------------")
            #print("attribute values: ")
            #print(attribute_values)
            #print("---------------")
            #print("---------------")

            tree = Tree_DPDT(attribute_indexes, 
                            A, # original style A
                            attribute_values, # new style attribute vals
                            root, 
                            class_values, 
                            feature_discrete, 
                            'IGGR',  # treetype
                            dataset_size, 
                            epsilon_per_tree,
                            max_depth) 

            tree.train(training_data[:len(training_data)//2], 1)
           
            # TO EVOLVE HERE 

            # attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

            #num_unclassified = tree.filter_training_data_and_count(training_data, epsilon_per_tree, class_values)
            
            #if num_unclassified > 0:
            #    print("number of unclassified records = {}".format(num_unclassified))
            

            print("currently pruning! ")
            # D2[:len(D2)//2], D2[len(D2)//2:]
            tree.prune(training_data[:len(training_data)//2], training_data[len(training_data)//2:])

            # FOREST = FOREST U TREE
            self._trees.append(tree)

        # print(self._trees)
        # print(len(self._trees))
        # print("succesfully ended building tree dictionary")


    def get_domains(self, data):
        attr_domains = {}
        transData = np.transpose(data)
        for i in range(0,len(data[0]) - 1):
            attr_domains[str(i)] = [str(x) for x in set(transData[i])]
            print("original domain length of categorical attribute {}: {}".format(i, len(attr_domains[str(i)])))
        return attr_domains


    def reduce_trees(self, num_trees, valid_attribute_sizes, dataset_size):
        num_trees -= 1
        largest_attribute = sorted(valid_attribute_sizes, key=lambda x:x[1], reverse=True)[0]
        #print("Removing att{} with domain size {}".format(largest_attribute[0], largest_attribute[1])) 
        new_valids = [ x for x in valid_attribute_sizes if x[0] != largest_attribute[0] ]
        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] ) # average with all attributes
        narrowed_average_domain = np.mean([x[1] for x in valid_attribute_sizes if x[0] in [y[0] for y in new_valids] ]) # average without the big attributes
        estimated_support_squared = dataset_size / (average_domain * narrowed_average_domain)
        return num_trees, new_valids, estimated_support_squared


    # GET MAJORITY LABELS (Forest, T, C) equivalent from pseudocode
    def evaluate_accuracy_with_voting(self, records, class_index=4):

        ''' Calculate the Prediction Accuracy of the Forest. '''
        actual_labels = [x[class_index] for x in records]

        predicted_labels = []
        leafs_not_used = 0
        count_of_averages_used = 0

        class_counts = defaultdict(list)

        # FOR EACH RECORD X IN DATASET DO
        # Strange for we need to access individual predictions here...
        for rec in records:

            h = []

            class_value_fractions = defaultdict(list)

            # FOR EACH TREE DO
            for tree in self._trees:

                # four trees
                # <__main__.Tree_DPDT object at 0x7f299b73c580>, 

                # GET PREDICTED CLASSIFICATION RESULT for a single record
                # node, leaf_not_used = tree._classify(tree._root_node, rec)

                result = tree.pred(np.array([rec]))

                h.append(*result)

                #unique, counts = np.unique(h, return_counts=True)
                #class_counts = dict(zip(unique, h))

                #print("class counts")
                #print(class_counts)

                #leafs_not_used += leaf_not_used

                #support = float(sum([v for k,v in noisy_class_counts.items()]))

                #for k,v in noisy_class_counts.items():
                #    class_value_fractions[k].append(v/support)
                    
            #best_confidences = {}

            #for k,lis in class_value_fractions.items():
            #   best_confidences[k] = max(lis)
            #best = [None, 0.0]

            #for k,class_best in best_confidences.items():
            #    if class_best > best[1]:
            #        best = [k, class_best]
            
            # average_used = False

            # majority vote
            #for k, class_best in best_confidences.items():

            #    if class_best == best[1] and k != best[0]:

                    # average_used = True
                   
                    #orig_average = np.mean(class_value_fractions[best[0]])

                    # contender_average = np.mean(class_value_fractions[k])
            
                    # if contender_average > orig_average:
            #            best = [k, class_best]

            # count_of_averages_used += 1 if average_used else 0

            # majority vote

            # IndexError: index 1 is out of bounds for axis 0 with size 1
            hh = [int(*el) for el in h]

            print("hh now contains: ")
            print(hh)

            c = Counter(hh)
            ans = c.most_common(1)

            predicted_labels.append(ans[0][0])

        print("data and trees looped trhough!")

        # counts how many records we got right!

        
        print(type(predicted_labels[0]))
        print(type(actual_labels[0]))

        # print(zip(predicted_labels, actual_labels))

        counts = Counter([x == y for x, y in zip(map(str, predicted_labels), actual_labels)])
        
        # acc = float(counts[True]) / len(records)
        
        return float(counts[True]) / len(records)
        

In [ ]:
D22

In [399]:

# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m

B = 0.2 # epsilon
f = 3
dm = 5

diff_priv_forest = DPRF_Forest(adult2, D2[400:], B, f, dm)

#num_trees = len(our_diff_priv_forest._trees)
#av_prunings = np.mean([x._prunings for x in our_diff_priv_forest._trees])
#av_tree_size = np.mean([x._id-x._prunings+1 for x in our_diff_priv_forest._trees])
#accuracy, leafs_not_most_confident, votes_requiring_average = diff_priv_forest.evaluate_accuracy_with_voting(adult23, class_index=0) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

original domain length of categorical attribute 0: 61
original domain length of categorical attribute 1: 16
original domain length of categorical attribute 2: 14
original domain length of categorical attribute 3: 2
NUM TREES = 4 & EPSILON PER TREE = 0.05
root attributes
[0, 1, 2, 3]
what is root attributes: 
[1, 2, 3]
count dictionary: 
{'1': 466.84339588106656, '0': 184.3607656707364}
most frequent : 
1
count dictionary: 
{'1': 461.13772668599006, '0': 175.4981224070171}
most frequent : 
1
count dictionary: 
{'1': 424.9857024178453, '0': 164.89169592455497}
most frequent : 
1
count dictionary: 
{'1': 389.50119267558864, '0': 160.40557086338973}
most frequent : 
1
count dictionary: 
{'1': 145.6166737067601, '0': 32.47553547693035}
most frequent : 
1
count dictionary: 
{'1': 48.19074715519977, '0': 6.66726412987955}
most frequent : 
1
count dictionary: 
{'1': 39.57316254132276, '0': 9.559360906094327}
most frequent : 
1
count dictionary: 
{'1': 41.4714794373186, '0': 10.251544025629801}

In [269]:
g = D2[400:][:,-1]
unique, counts = np.unique(g, return_counts=True)
print(np.asarray((unique, counts)).T)

[['0' '308']
 ['1' '920']]


In [400]:
accuracy = diff_priv_forest.evaluate_accuracy_with_voting(D2[400:][:,0:5], class_index=4) # confidence count includes all trees (before voting)
print("accuracy = {}".format(accuracy))

i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  0
i:  0
i:  0
hh now contains: 
[1, 1, 1, 1]
i:  0
i:  